In [4]:
!pip install tweepy
!pip install textblob
!python -m textblob.download_corpora

    100% |████████████████████████████████| 133kB 720kB/s ta 0:00:01
You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 634kB 983kB/s ta 0:00:011
You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
[nltk_data] Downloading package brown to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /home/jovyan/nltk_

In [11]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
 
class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = 'sUHh7JRlyO8FyR13IDeA8fU9i'
        consumer_secret = 'eop24iZFggjofseChtXMeE9rzqrcYAI7ygAj2WhXDNRhL2yCNF'
        access_token = '841161497254420480-mJgm5iUzsZYbj4GOALwrAUJU6pKjA4l'
        access_token_secret = '0HcN005SPyZ8iXnCdzPVTPn2EEYsU2MjbffEiDEiExIPg'
 
        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")
 
    #def clean_tweet(self, tweet):
     #   '''
      #  Utility function to clean tweet text by removing links, special characters
       # using simple regex statements.
        #'''
        #return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])
         #                           |(\w+:\/\/\S+)", " ", tweet).split())
 
    #def get_tweet_sentiment(self, tweet):
   #     '''
    #    Utility function to classify sentiment of passed tweet
     #   using textblob's sentiment method
      #  '''
        # create TextBlob object of passed tweet text
       # analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        #if analysis.sentiment.polarity > 0:
         #   return 'positive'
        #elif analysis.sentiment.polarity == 0:
         #   return 'neutral'
        #else:
         #   return 'negative'
 
    def get_tweets(self, query, count = 10):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []
 
        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = count)
 
            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                
                print(tweet.text)
#                 parsed_tweet = {}
 
#                 # saving text of tweet
#                 parsed_tweet['text'] = tweet.text
#                 # saving sentiment of tweet
#                 parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
 
#                 # appending parsed tweet to tweets list
#                 if tweet.retweet_count > 0:
#                     # if tweet has retweets, ensure that it is appended only once
#                     if parsed_tweet not in tweets:
#                         tweets.append(parsed_tweet)
#                 else:
#                     tweets.append(parsed_tweet)
 
            # return parsed tweets
            return tweets
 
        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))
 
def main():
    # creating object of TwitterClient Class
    api = TwitterClient()
    # calling function to get tweets
    tweets = api.get_tweets(query = 'siddramaiah', count = 200)
 
    # picking positive tweets from tweets
#     ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
#     # percentage of positive tweets
#     print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
#     # picking negative tweets from tweets
#     ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
#     # percentage of negative tweets
#     print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
#     # percentage of neutral tweets
#     print("Neutral tweets percentage: {} % \
#         ".format(100*len(tweets - ntweets - ptweets)/len(tweets)))
 
#     # printing first 5 positive tweets
#     print("\n\nPositive tweets:")
#     for tweet in ptweets[:10]:
#         print(tweet['text'])
 
#     # printing first 5 negative tweets
#     print("\n\nNegative tweets:")
#     for tweet in ntweets[:10]:
#         print(tweet['text'])
 
if __name__ == "__main__":
    # calling main function
    main()


BJP -JDS ಸವಾಲಿಗೆ ಹೆದರಿದರಾ CM ? : ಒಕ್ಕಲಿಗರ ಓಲೈಕೆಗೆ ಮುಂದಾದ ಸಿದ್ದರಾಮಯ್ಯ - https://t.co/Vey3BZaVNE https://t.co/Neajjns6PB
@INCIndia @JhaSanjay @RahulGandhi PCghotala, veerbhadraghotala, Vadraghotala, national heraldghotala,  kcvenugopal… https://t.co/ezZ4zFm7yl
RT @Satyanewshi: Karnataka CM Siddramaiah  was hailed in intellectual circles for passing the Anti-Superstition bill.  IN reality 
1)  Sidd…
RT @Satyanewshi: Karnataka CM Siddramaiah  was hailed in intellectual circles for passing the Anti-Superstition bill.  IN reality 
1)  Sidd…
Karnataka CM Siddramaiah  was hailed in intellectual circles for passing the Anti-Superstition bill.  IN reality 
1… https://t.co/HTlNduD39E
@siddaramaiah Siddramaiah is against superstition not against tradition. Superstition is irrational and can harm th… https://t.co/7AxSRgD3eX
#EXCLUSIVE_PICTURES
Today Namma #KichchaSudeepa Anna Met Honorable CM of Karnataka #Siddramaiah Sir!

@KicchaSudeep https://t.co/Lm1XV4Ngnd
@Gaddadaari @vinod_gowda Correct agi 